# Constellation and Chain Analysis: MultiConstellation Multihop

<img src="MultipleConstellations.jpg" alt="Drawing" style="width: 500px;"/>

**Terminology**
* Node = Object in STK
* Edge = Access between two objects in STK
* Strand = The sequence of nodes and edges to complete access in a chain

**This notebook shows how to:**
* Find the shortest path between a starting and ending constellation, with many potential intermediate constellations.
* A chain will be built between each sequential pair in the constellationOrder list. Then networkx will be used build the network with the nodes coming from the constellations and the connections between the nodes coming from the chain accesses. Multiple sublists can be passed into constellationOrderList.
* To reduce the runtime on subsequent runs, the results can be saved to binary files and loaded in for later use. The strands from the chains will be saved in the SavedNodes folder, the nodes and associated time delays will be saved in the SavedNodes folder, the node positions over time are saved in the SavedPositions folder and the accesses between nodes over time are saved in the SavedEdges folder. These folders will be created as subfolders of the directory used to run the script. During the first run the files will automatically be built and saved, subsequent runs will reload these files. To make changes simply delete the associated .pkl file for any changed strands, nodes, etc. and the script will recompute the data as needed. Or force all of the data to be overridden by setting the override options to be True.
* Typical STK constraints such as range, link duration, Eb/No, etc are taken into account
* Data in the df variable can be pushed back into STK as a user supplied variable, a strand can be shown using object lines, and active objects over the analysis time period or at a time instance can be turned on.


In [ ]:
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 120)
from comtypes.client import CreateObject
from comtypes.client import GetActiveObject
from comtypes.gen import STKObjects
from comtypes.gen import STKUtil
from comtypes.gen import AgSTKVgtLib
import seaborn as sns
import matplotlib.pyplot as plt
from chainPathLib2 import *
import time
import networkx as nx
folders = ['SavedNodes','SavedPositions','SavedStrands','SavedEdges','SavedNetworkData']
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

## Constellation Connection Order, Computation Time, Metric, Saved Data Options

In [28]:
constellationOrderLists = [['Targets','ObservingSatsFORs','ObservingSats','ObservingSatsTransmitters','ObservingSatsReceivers','ObservingSats','RelaySatsFORs','RelaySats','RelaySatsFORs','EndLocations'],['RelaySats','RelaySats']]   
startingConstellation = 'Targets'
endingConstellation = 'EndLocations'

start = 0 # EpSec
stop = 60*10 # EpSec
metric = 'timeDelay' # 'distance' or 'timeDelay'
nodeDelays = {'ObservingSatsFORs':0.01,'ObservingSatsTransmitters':0.005,'ObservingSatsReceivers':0.005,'RelaySatsFORs':0.01,'RelaySats':0.002} # Add in time delays. Provide the constellation name in STK and the node delays
stkVersion = 12
overrideStrands = False # Override previously computed chains
overrideNodeDelaysByNode = False # Override previously built node delay dictionaries
overrideNodesTimesPos = False # Override previously built node positions
overrideNetwork = False # Override previously built node positions

In [30]:
# Connect to STK
stkApp = GetActiveObject('STK{}.Application'.format(stkVersion))
stkRoot = stkApp.Personality2
stkRoot.Isolate()
stkRoot.UnitPreferences.SetCurrentUnit('DateFormat','EpSec') # Units to EpSec for ease of use
stkRoot.ExecuteCommand('Units_SetConnect / Date "EpochSeconds"');
stkRoot.ExecuteCommand('VO * ObjectLine DeleteAll'); # Clean up old object lines

# Build chains and create a dict of time delays for each node
t1 = time.time()
chainNames = createDirectedChains(stkRoot,constellationOrderLists,start=start,stop=stop,color=12895232)   
print(time.time()-t1)
t1 = time.time()
nodeDelaysByNode = getNodeDelaysByNode(stkRoot,nodeDelays,chainNames=chainNames,overrideData=overrideNodeDelaysByNode)
print(time.time()-t1)

0.6350052356719971
0.00400090217590332


## Compute  Strands and Distances

In [4]:
# Compute strands
t1 = time.time()
strands,dfStrands = getAllStrands(stkRoot,chainNames,start,stop,overrideData=overrideStrands)
print(time.time()-t1)
dfStrands

0.15405750274658203


,strand,start,stop,dur,num hops
827,"(Target/Target3, Satellite/ObservingSat63/Sensor/ObservingFOR)",0.000000,9.216490,9.216490,0.0
33844,"(Satellite/ObservingSat53, Satellite/RelaySat22/Sensor/RelaySatFOR)",0.000000,13.059304,13.059304,0.0
34466,"(Satellite/ObservingSat56, Satellite/RelaySat24/Sensor/RelaySatFOR)",0.000000,13.059304,13.059304,0.0
31332,"(Satellite/ObservingSat33, Satellite/RelaySat12/Sensor/RelaySatFOR)",0.000000,13.059306,13.059306,0.0
31954,"(Satellite/ObservingSat36, Satellite/RelaySat14/Sensor/RelaySatFOR)",0.000000,13.059306,13.059306,0.0
...,...,...,...,...,...
8178,"(Satellite/ObservingSat25/Transmitter/ObservingTransmitter, Satellite/ObservingSat63/Receiver/ObservingReceiver)",86377.072809,86400.000000,22.927191,0.0
9963,"(Satellite/ObservingSat32/Transmitter/ObservingTransmitter, Satellite/ObservingSat76/Receiver/ObservingReceiver)",86377.072809,86400.000000,22.927191,0.0
11505,"(Satellite/ObservingSat35/Transmitter/ObservingTransmitter, Satellite/ObservingSat73/Receiver/ObservingReceiver)",86377.072809,86400.000000,22.927191,0.0
1000,"(Target/Target4, Satellite/ObservingSat16/Sensor/ObservingFOR)",86389.459523,86400.000000,10.540477,0.0


In [5]:
# Compute node positions, distances and time delays

# Time resolution of distance/time computation
step = 10 # sec

t1 = time.time()
nodesTimesPos = computeNodesPosOverTime(stkRoot,strands,start,stop,step,overrideData=overrideNodesTimesPos) # Pull node position over time
t2 = time.time()
print(t2-t1)

t1 = time.time()
strandsAtTimes = getStrandsAtTimes(strands,start,stop,step) # Discretize strand intervals into times
t2 = time.time()
print(t2-t1)

t1 = time.time()
timeNodePos = computeTimeNodePos(strandsAtTimes,nodesTimesPos) # Nodes and positions at each time
t2 = time.time()
print(t2-t1)

t1 = time.time()
timesEdgesDistancesDelays = computeTimeEdgesDistancesDelays(strandsAtTimes,nodesTimesPos,nodeDelaysByNode,overrideData=True)  # Edges, distances and delays at each time
t2 = time.time()
print(t2-t1)

1.1173458099365234
41.17776823043823
8.21703052520752
118.07205438613892


## Use NX for Network Metrics and Reliability Analysis


In [7]:
# Get pairs of each starting and ending node permutation in the constellations
startingNodes = getNodesFromConstellation(stkRoot,startingConstellation)
endingNodes = getNodesFromConstellation(stkRoot,endingConstellation)
nodePairs = [(start,end) for start,end in itertools.product(startingNodes, endingNodes)] # full permutation
pd.DataFrame(nodePairs)

,0,1
0,Target/Target1,Place/Los_Angeles_CA
1,Target/Target1,Place/Washington_DC
2,Target/Target2,Place/Los_Angeles_CA
3,Target/Target2,Place/Washington_DC
4,Target/Target3,Place/Los_Angeles_CA
5,Target/Target3,Place/Washington_DC
6,Target/Target4,Place/Los_Angeles_CA
7,Target/Target4,Place/Washington_DC
8,Target/Target5,Place/Los_Angeles_CA
9,Target/Target5,Place/Washington_DC


In [8]:
# Loop through each node pair and compute network metrics
# Edit computeNetworkMetrics in chainPathLibCustom to for additional metrics
for nodePair in nodePairs:
    df = computeNetworkMetrics(start,stop,step,timeNodePos,timesEdgesDistancesDelays,[nodePair[0]],[nodePair[1]],metric,computeNumNodesToLoseAccessBetweenAnyPair=True,overrideData=overrideNetwork,printTime=False,diNetwork=True)

##  Investigate Routing between Nodes

In [31]:
# Pick a starting and ending node
startingNode = 'Target/Target7'
endingNode = 'Place/Washington_DC'

In [32]:
# Load df
filename = 'SavedNetworkData/df{}{}.pkl'.format(startingNode.split('/')[-1],endingNode.split('/')[-1])
with open(filename,'rb') as f:
    df = pickle.load(f)
df = addLightAndNodeDelays(df,timesEdgesDistancesDelays)    
dfIntervals = createDfIntervals(df,stop,step)
addStrandsAsObjectLines(stkRoot,dfIntervals,color='yellow')

In [33]:
# Add data back into STK for reporting and plotting
t1 = time.time()
df['distance'] = df['distance']*1000 # May need to fix meter/kilometer issue
addDataToSTK(stkRoot,chainNames[0],df) # Adds data in df back into STK to the first chain under User Supplied data
print(time.time()-t1)

14.579044818878174


In [34]:
df

,time,strand,timeDelay,num hops,num parent hops,Highest Num Nodes Removed To Lose Access,Lowest Num Nodes Removed To Lose Access,distance,lightDelay,nodeDelay
0,0,"[Target/Target7, Satellite/ObservingSat21/Sensor/ObservingFOR, Satellite/ObservingSat21, Satellite/ObservingSat21/Tr...",0.108642,6.0,3.0,3,3,2.357617e+07,0.078642,0.03
1,10,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",0.108554,6.0,3.0,3,3,2.354994e+07,0.078554,0.03
2,20,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",0.108469,6.0,3.0,3,3,2.352454e+07,0.078469,0.03
3,30,"[Target/Target7, Satellite/ObservingSat21/Sensor/ObservingFOR, Satellite/ObservingSat21, Satellite/ObservingSat21/Tr...",0.108504,6.0,3.0,3,3,2.353483e+07,0.078504,0.03
4,40,"[Target/Target7, Satellite/ObservingSat21/Sensor/ObservingFOR, Satellite/ObservingSat21, Satellite/ObservingSat21/Tr...",0.108473,6.0,3.0,3,3,2.352567e+07,0.078473,0.03
...,...,...,...,...,...,...,...,...,...,...
8636,86360,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",0.112834,6.0,3.0,1,1,2.483314e+07,0.082834,0.03
8637,86370,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",0.112952,6.0,3.0,1,1,2.486833e+07,0.082952,0.03
8638,86380,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",0.113069,6.0,3.0,1,1,2.490357e+07,0.083069,0.03
8639,86390,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",0.113187,6.0,3.0,1,1,2.493886e+07,0.083187,0.03


In [35]:
dfIntervals

,start,stop,dur,strand,num hops
0,0.0,5.0,5.0,"[Target/Target7, Satellite/ObservingSat21/Sensor/ObservingFOR, Satellite/ObservingSat21, Satellite/ObservingSat21/Tr...",6.0
1,5.0,25.0,20.0,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",6.0
3,25.0,55.0,30.0,"[Target/Target7, Satellite/ObservingSat21/Sensor/ObservingFOR, Satellite/ObservingSat21, Satellite/ObservingSat21/Tr...",6.0
6,55.0,65.0,10.0,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",6.0
7,65.0,135.0,70.0,"[Target/Target7, Satellite/ObservingSat36/Sensor/ObservingFOR, Satellite/ObservingSat36, Satellite/ObservingSat36/Tr...",6.0
...,...,...,...,...,...
8551,85505.0,85865.0,360.0,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",6.0
8587,85865.0,85885.0,20.0,"[Target/Target7, Satellite/ObservingSat36/Sensor/ObservingFOR, Satellite/ObservingSat36, Satellite/ObservingSat36/Tr...",6.0
8589,85885.0,86235.0,350.0,"[Target/Target7, Satellite/ObservingSat36/Sensor/ObservingFOR, Satellite/ObservingSat36, Satellite/ObservingSat36/Tr...",6.0
8624,86235.0,86315.0,80.0,"[Target/Target7, Satellite/ObservingSat36/Sensor/ObservingFOR, Satellite/ObservingSat36, Satellite/ObservingSat36/Tr...",6.0


In [36]:
# Active objects in the network over time
objPaths = list(set((item for sublist in df['strand'] for item in sublist)))

In [37]:
# Turn on the objects in the scenario
turnGraphicsOnOff(stkRoot,objPaths,onOrOff = 'On',parentsOnly = True)

In [38]:
# Turn off the objects in the sceario
turnGraphicsOnOff(stkRoot,objPaths,onOrOff = 'Off',parentsOnly = True)

## Investigate Instances in Time

In [39]:
# Look at an instance in time (pick a time in df)
t = 0
stkRoot.CurrentTime = t

In [40]:
# Look at strand order and the node delay
objPaths = df['strand'][t/step]
nodeDelaysByStrand = {node:nodeDelaysByNode[node] for node in objPaths}
pd.DataFrame([*nodeDelaysByStrand.items()],columns=['node','nodeDelay'])

,node,nodeDelay
0,Target/Target7,0.000
1,Satellite/ObservingSat21/Sensor/ObservingFOR,0.010
2,Satellite/ObservingSat21,0.000
3,Satellite/ObservingSat21/Transmitter/ObservingTransmitter,0.005
4,Satellite/ObservingSat83/Receiver/ObservingReceiver,0.005
5,Satellite/ObservingSat83,0.000
6,Satellite/RelaySat22/Sensor/RelaySatFOR,0.010
7,Place/Washington_DC,0.000


In [41]:
#look at possible connections for the object of interest at that time
nodeInterest = objPaths[0]
possibleNodeConnections(t,nodeInterest,timesEdgesDistancesDelays)

,node1,node2,distance,timeDelay
0,Target/Target7,Satellite/ObservingSat55/Sensor/ObservingFOR,3393.680533,0.021320
1,Target/Target7,Satellite/ObservingSat21/Sensor/ObservingFOR,3427.049297,0.021431
2,Target/Target7,Satellite/ObservingSat64/Sensor/ObservingFOR,3427.049297,0.021431
3,Target/Target7,Satellite/ObservingSat36/Sensor/ObservingFOR,3166.822072,0.020563


In [42]:
# Turn on the objects in the scenario
turnGraphicsOnOff(stkRoot,objPaths,onOrOff = 'On',parentsOnly = False)

In [21]:
# Turn off the objects in the sceario
turnGraphicsOnOff(stkRoot,objPaths,onOrOff = 'Off',parentsOnly = False)

## Investigate Node Utilization

In [22]:
# Most frequnt node in the shortest path and the sum of their durations
strands = dfIntervals[['strand','start','stop']].values
dfNodesIntervals = getNodesIntervalsFromStrands(strands)
dfNodeActive = getActiveDuration(dfNodesIntervals,start,stop)
dfNodeActive.sort_values('sum dur',ascending=False).head(10)

,sum dur,% time active
node,,
Place/Washington_DC,82020.0,94.930556
Target/Target7,82020.0,94.930556
Satellite/RelaySat34/Sensor/RelaySatFOR,6680.0,7.731481
Satellite/RelaySat31/Sensor/RelaySatFOR,6230.0,7.210648
Satellite/RelaySat21/Sensor/RelaySatFOR,6220.0,7.199074
Satellite/RelaySat44/Sensor/RelaySatFOR,6200.0,7.175926
Satellite/RelaySat11/Sensor/RelaySatFOR,6170.0,7.141204
Satellite/RelaySat42/Sensor/RelaySatFOR,6110.0,7.071759
Satellite/RelaySat24/Sensor/RelaySatFOR,6020.0,6.967593
